# Training Model Using Deep Learning


### Tensor2Tensor(T2T)
an open-source system for training deep learning models in TensorFlow.  
A deep learning system consists of datasets, model architectures, optimizers, and hyperparameters.Tensor2tensor ties together all these pieces in coherent and standarized way.

In [4]:
# We want to create a machine learning model that completes javascript codes.

# Creating a dataset

In [5]:
# Writing a script that parse the text from mlab and split it into two files input.txt and output.txt
import json

with open("wikipedia-JSCoverage.json") as f1:
	json_obj = json.loads(f1.read())
	# print(json_obj)
	with open("raw.txt", "w") as f2:
		for junk in json_obj["junk"]:
			for junklist in junk["junk_list"]:
				f2.write(junklist + "\n");

	# print(json_obj["junk"])


with open("raw.txt") as rawf, \
	 open("input.txt", "w") as inf, \
	 open("output.txt", "w") as outf:
	prev_line = ''
	for curr_line in rawf:
		curr_line = curr_line.strip()
		if len(prev_line) > 0 and len(curr_line) > 0:
			inf.write(prev_line)
			outf.write(curr_line)
		prev_line = curr_line



# Defining a problem

In [7]:
%%writefile problem.py
import os
import tensorflow as tf
from tensor2tensor.utils import registry
from tensor2tensor.models import transformer
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import generator_utils


@registry.register_problem
class JavascriptProblem(text_problems.Text2TextProblem):

  @property
  def approx_vocab_size(self):
    return 2**13  # ~8k

  @property
  def is_generate_per_split(self):
    # generate_data will NOT shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 10% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 90,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 10,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    with open('raw.txt', 'r') as rawfp:
      prev_line = ''
      for curr_line in rawfp:
        curr_line = curr_line.strip()
        if len(prev_line) > 0 and len(curr_line) > 0:       
            yield {
                "inputs": prev_line,
                "targets": curr_line
            }
        prev_line = curr_line          


# Smaller than the typical translate model, and with more regularization
@registry.register_hparams
def transformer_javascript():
  hparams = transformer.transformer_base()
  hparams.num_hidden_layers = 2
  hparams.hidden_size = 128
  hparams.filter_size = 512
  hparams.num_heads = 4
  hparams.attention_dropout = 0.6
  hparams.layer_prepostprocess_dropout = 0.6
  hparams.learning_rate = 0.05
  return hparams

# hyperparameter tuning ranges
@registry.register_ranged_hparams
def transformer_javascript_range(rhp):
  rhp.set_float("learning_rate", 0.05, 0.25, scale=rhp.LOG_SCALE)
  rhp.set_int("num_hidden_layers", 2, 4)
  rhp.set_discrete("hidden_size", [128, 256, 512])
  rhp.set_float("attention_dropout", 0.4, 0.7)

Writing problem.py


# Running Model

In [ ]:
# --problem=languagemodel

pip install tensor2tensor && t2t-trainer \
--generate_data \
--data_dir=~/t2t_data \
--output_dir=~/t2t_train/mnist \
--problem=languagemodel_ptb10k \
--model=transformer \
--hparams_set=transformer_small \
--train_steps=200 \
--eval_steps=100

t2t-trainer \
  --generate_data \
  --data_dir=~/_programming/major_project/deep_learning/t2t_data \
  --output_dir=~/_programming/major_project/deep_learning/t2t_train/mnist \
  --problem=$PROBLEM \
  --model=transformer \
  --hparams_set=transformer_small \
  --train_steps=1000 \
  --eval_steps=100

# Decoding model

In [ ]:
t2t-decoder \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --model=$MODEL \
  --hparams_set=$HPARAMS \
  --output_dir=$TRAIN_DIR \
  --decode_hparams="beam_size=$BEAM_SIZE,alpha=$ALPHA" \
  --decode_from_file=$DECODE_FILE \
  --decode_to_file=translation.en

# run this, before running make decode_this.txt with Hello, world etc.

t2t-decoder \
  --data_dir=$HOME/t2t_data \
  --problem=languagemodel_ptb10k \
  --model=transformer \
  --hparams_set=transformer_small \
  --output_dir=$HOME/t2t_train/languagemodel_ptb10k/transformer-transformer_small \
  --decode_hparams="beam_size=4,alpha=0.4" \
  --decode_from_file=$HOME/t2t_data/decode_this.txt \
  --decode_to_file=translation.en

# see the file
cat translation.en